<a href="https://colab.research.google.com/github/RenatoChavezCajahuanca/Wind_energy_Peru/blob/main/Vajont_Peru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Definir el Objetivo:
Comparar los deslizamientos de tierra en Vajont y Yungay, Perú, mediante simulaciones y análisis visuales para identificar similitudes y diferencias en sus causas, dinámicas y efectos.
2. Estrategia de Desarrollo:
Paso 1: Investigación y Recopilación de Datos
Buscar datos precisos sobre los deslizamientos de Vajont y Yungay, incluyendo:

Tipos de suelo.
Dinámicas de movimiento.
Características geológicas (arcillas, grietas, niveles freáticos).
Factores desencadenantes (lluvias, terremotos, presión hidrostática).
Usar fuentes confiables:

Artículos científicos y estudios históricos.
GitHub: Buscar repositorios relacionados con datos geológicos y deslizamientos.
Organismos oficiales: Institutos geológicos (USGS, INGEMMET).
Paso 2: Análisis Preliminar de Datos
Procesar los datos:

Convertir archivos CSV, GeoJSON, o SHP a un formato compatible con Python.
Preprocesar usando bibliotecas como pandas y numpy.
Visualizar los datos iniciales:

Utilizar matplotlib y seaborn para entender las relaciones clave entre las variables (e.g., altura vs. volumen desplazado).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit


In [ ]:
import shutil

# Eliminar la carpeta existente
shutil.rmtree("FW_Vajont_2019")

print("Carpeta eliminada. Ahora podemos clonar el repositorio nuevamente.")


In [ ]:
!git clone https://github.com/fischanger/FW_Vajont_2019.git


In [ ]:
import os

# Listar contenido principal del repositorio
contenido_principal = os.listdir("FW_Vajont_2019")
print("Contenido del repositorio:", contenido_principal)


In [ ]:
# Listar el contenido de la carpeta DTM
contenido_dtm = os.listdir("FW_Vajont_2019/DTM")
print("Contenido de la carpeta DTM:", contenido_dtm)


In [ ]:
# Instalar py7zr para manejar archivos .7z
!pip install py7zr


In [ ]:
import py7zr

# Ruta del archivo .7z
ruta_dtm_7z = "FW_Vajont_2019/DTM/dtm.2x2.area2.xyz.7z"

# Ruta para extraer los datos
ruta_dtm_extraida = "FW_Vajont_2019/DTM/extraida"

# Extraer el contenido
with py7zr.SevenZipFile(ruta_dtm_7z, mode='r') as z:
    z.extractall(path=ruta_dtm_extraida)

# Listar los archivos extraídos
contenido_extraido = os.listdir(ruta_dtm_extraida)
print("Contenido extraído de DTM:", contenido_extraido)


In [ ]:
# Ruta del archivo extraído
ruta_xyz = os.path.join(ruta_dtm_extraida, "dtm.2x2.area2.xyz")

# Leer el archivo .xyz
import pandas as pd

# Cargar datos (separados por espacios o tabulaciones)
dtm_data = pd.read_csv(ruta_xyz, delim_whitespace=True, header=None, names=["X", "Y", "Z"])

# Mostrar las primeras filas
print("Primeras filas del archivo DTM:")
print(dtm_data.head())


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Crear figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Graficar el modelo digital del terreno
ax.scatter(dtm_data["X"], dtm_data["Y"], dtm_data["Z"], c=dtm_data["Z"], cmap='terrain', s=1)

# Etiquetas y título
ax.set_title("Modelo Digital del Terreno (DTM)")
ax.set_xlabel("Coordenada X")
ax.set_ylabel("Coordenada Y")
ax.set_zlabel("Elevación Z")

plt.show()


In [ ]:
import pandas as pd
import os

# Ruta del archivo descomprimido (ajusta según tu estructura de archivos)
ruta_archivo_xyz = "FW_Vajont_2019/DTM/extraida/dtm.2x2.area2.xyz"

# Cargar datos desde el archivo .xyz
try:
    dtm_data = pd.read_csv(ruta_archivo_xyz, delim_whitespace=True, header=None, names=["X", "Y", "Z"])
    print("Archivo cargado exitosamente. Primeras filas:")
    print(dtm_data.head())
except FileNotFoundError:
    print(f"Error: El archivo '{ruta_archivo_xyz}' no se encuentra. Revisa la ruta.")


In [ ]:
import numpy as np
from scipy.interpolate import griddata

# Crear mallas para X, Y y Z
grid_x, grid_y = np.meshgrid(
    np.linspace(dtm_data["X"].min(), dtm_data["X"].max(), 100),
    np.linspace(dtm_data["Y"].min(), dtm_data["Y"].max(), 100)
)

# Interpolación de los valores de Z
grid_z = griddata(
    (dtm_data["X"], dtm_data["Y"]),  # Puntos originales
    dtm_data["Z"],                  # Valores de Z originales
    (grid_x, grid_y),               # Puntos de la malla
    method="linear"                 # Método de interpolación: 'linear', 'nearest', 'cubic'
)

print("Mallas creadas exitosamente.")
print(f"Dimensiones de grid_x: {grid_x.shape}")
print(f"Dimensiones de grid_y: {grid_y.shape}")
print(f"Dimensiones de grid_z: {grid_z.shape}")


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Crear la figura interactiva con Plotly
fig = go.Figure()

# Añadir la superficie a la figura
# Usar una colorescale válida de Plotly (ejemplo: 'Viridis')
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',  # Colorescale válida
    colorbar=dict(title="Elevación Z (metros)"),
    showscale=True
))

# Configurar etiquetas y título
fig.update_layout(
    title="Modelo Digital del Terreno (Interactivo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar la figura interactiva
fig.show()


In [ ]:
# Ruta del archivo CSV de salida
ruta_csv = "modelo_digital_terreno.csv"

# Guardar los datos en formato CSV
try:
    dtm_data.to_csv(ruta_csv, index=False)
    print(f"Modelo digital del terreno guardado como: {ruta_csv}")
except Exception as e:
    print(f"Error al guardar el archivo CSV: {e}")


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Crear la figura interactiva con Plotly
fig = go.Figure()

# Añadir la superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)"),
    showscale=True
))

# Anotaciones en el espacio 3D
anotaciones = [
    dict(
        x=grid_x[50, 50], y=grid_y[50, 50], z=grid_z[50, 50],  # Coordenadas dentro del rango
        text="Área de desplazamiento",                         # Texto de la anotación
        showarrow=True,
        arrowhead=2,
        ax=0, ay=0,                                            # Ajuste de la flecha
        arrowcolor="black",
        font=dict(color="black", size=12, family="Arial")
    ),
    dict(
        x=grid_x[99, 99], y=grid_y[99, 99], z=grid_z[99, 99],  # Último índice válido
        text="Zona estable",
        showarrow=True,
        arrowhead=2,
        ax=0, ay=0,
        arrowcolor="red",
        font=dict(color="red", size=12, family="Arial")
    )
]

# Añadir las anotaciones a la escena
fig.update_layout(
    title="Modelo Digital del Terreno con Leyendas",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z",
        annotations=anotaciones  # Mover anotaciones a la escena
    )
)

# Mostrar la figura interactiva
fig.show()


In [ ]:
# Calcular estadísticas básicas de la elevación
print("Estadísticas de elevación (Z):")
print(f"Mínimo: {dtm_data['Z'].min()} metros")
print(f"Máximo: {dtm_data['Z'].max()} metros")
print(f"Promedio: {dtm_data['Z'].mean()} metros")

# Buscar valores que podrían indicar zonas de desprendimiento
# Por ejemplo, valores significativamente bajos
zona_desprendimiento = dtm_data[dtm_data['Z'] < dtm_data['Z'].mean() - 100]
print(f"Número de puntos con anomalías: {len(zona_desprendimiento)}")
print("Ejemplo de puntos con posibles desprendimientos:")
print(zona_desprendimiento.head())


In [ ]:
import plotly.graph_objects as go

# Crear una máscara para las zonas de desprendimiento (valores bajos de Z)
zona_desprendimiento = grid_z < grid_z.mean() - 100  # Ajusta el umbral según tus datos

# Crear la figura
fig = go.Figure()

# Superficie principal del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)"),
    showscale=True
))

# Superponer las zonas de desprendimiento en otro color (ejemplo: rojo)
fig.add_trace(go.Surface(
    z=grid_z,  # Mismo terreno
    x=grid_x,
    y=grid_y,
    surfacecolor=zona_desprendimiento.astype(int),  # Resaltar zonas desplazadas
    colorscale=[[0, 'rgba(0, 0, 0, 0)'], [1, 'red']],  # Colores: transparente y rojo
    showscale=False
))

# Título y etiquetas
fig.update_layout(
    title="Modelo Digital del Terreno con Zonas Desplazadas",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar la figura interactiva
fig.show()


In [ ]:
import matplotlib.pyplot as plt

# Crear la figura
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Superficie del terreno
surf = ax.plot_surface(grid_x, grid_y, grid_z, cmap='terrain', edgecolor='none')

# Añadir puntos de las zonas de desprendimiento
ax.scatter(
    grid_x[zona_desprendimiento], grid_y[zona_desprendimiento], grid_z[zona_desprendimiento],
    color='red', label='Zonas Desprendidas', s=10
)

# Etiquetas
ax.set_title("Modelo Digital del Terreno con Zonas Desplazadas")
ax.set_xlabel("Coordenada X")
ax.set_ylabel("Coordenada Y")
ax.set_zlabel("Elevación Z")

# Barra de color y leyenda
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
cbar.set_label("Elevación Z (metros)")
plt.legend()

plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Crear la figura interactiva con Plotly
fig = go.Figure()

# Identificar zonas desplazadas: valores de Z más bajos que el promedio menos un umbral
umbral_desprendimiento = 100  # Ajusta este valor según el análisis de tus datos
zona_desprendimiento = grid_z < (grid_z.mean() - umbral_desprendimiento)  # Máscara booleana

# Añadir la superficie principal del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)"),
    showscale=True,
    opacity=0.8  # Hacer la superficie semitransparente para mejor visualización
))

# Añadir las zonas desplazadas en color rojo
fig.add_trace(go.Surface(
    z=grid_z,  # Mismo terreno
    x=grid_x,
    y=grid_y,
    surfacecolor=zona_desprendimiento.astype(int),  # 1 para zonas desplazadas
    colorscale=[[0, 'rgba(0, 0, 0, 0)'], [1, 'red']],  # Transparente y rojo
    showscale=False,  # No mostrar la escala de color para las zonas
    opacity=1  # Totalmente opaco para resaltar
))

# Configurar etiquetas y título
fig.update_layout(
    title="Modelo Digital del Terreno con Zonas Desplazadas (Rojo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar la figura interactiva
fig.show()


In [ ]:
# Coordenadas calculadas o ajustadas según el sistema del modelo
presa_x = 2316500  # Ejemplo aproximado basado en el modelo
presa_y = 5127500
presa_z = grid_z.min() + 50  # Ajustar según la elevación mínima del modelo

# Añadir marcador al modelo interactivo
fig.add_trace(go.Scatter3d(
    x=[presa_x], y=[presa_y], z=[presa_z],
    mode='markers+text',
    marker=dict(size=10, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))


In [ ]:
print("Rangos del modelo:")
print(f"Rango de X: {grid_x.min()} a {grid_x.max()}")
print(f"Rango de Y: {grid_y.min()} a {grid_y.max()}")
print(f"Rango de Z: {grid_z.min()} a {grid_z.max()}")


In [ ]:
# Coordenadas ajustadas para la presa
presa_x = 2316500  # Cercano al límite superior de X
presa_y = 5127500  # Cercano al límite superior de Y
presa_z = grid_z.min() + 50  # Elevación baja para mayor visibilidad


In [ ]:
# Añadir marcador para la presa al modelo interactivo
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Mostrar el modelo
fig.show()


In [ ]:
import plotly.graph_objects as go

# Coordenadas ajustadas
presa_x = grid_x[-1, -1]
presa_y = grid_y[-1, -1]
presa_z = grid_z[-1, -1] + 50

# Crear figura interactiva
fig = go.Figure()

# Añadir superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Añadir un único marcador para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Mostrar el modelo
fig.update_layout(
    title="Modelo Digital del Terreno con la Presa de Vajont",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)
fig.show()


In [ ]:
import plotly.graph_objects as go

# Coordenadas ajustadas
presa_x = grid_x[-1, -1]
presa_y = grid_y[-1, -1]
presa_z = grid_z[-1, -1] + 50

# Crear la figura interactiva
fig = go.Figure()

# Añadir superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Añadir un único marcador para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Configurar el diseño
fig.update_layout(
    title="Modelo Digital del Terreno con la Presa de Vajont",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo
fig.show()


In [ ]:
# Coordenadas actuales de la presa
print("Coordenadas del marcador de la presa:")
print(f"Presa X: {presa_x}")
print(f"Presa Y: {presa_y}")
print(f"Presa Z: {presa_z}")

# Rango del modelo
print("Rango del modelo:")
print(f"Rango de X: {grid_x.min()} a {grid_x.max()}")
print(f"Rango de Y: {grid_y.min()} a {grid_y.max()}")
print(f"Rango de Z: {grid_z.min()} a {grid_z.max()}")


In [ ]:
# Visualizar las primeras filas de grid_x, grid_y, grid_z
print("Primeros valores de grid_x:")
print(grid_x[:5, :5])

print("Primeros valores de grid_y:")
print(grid_y[:5, :5])

print("Primeros valores de grid_z:")
print(grid_z[:5, :5])


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Coordenadas ajustadas de la presa
presa_x = grid_x[-1, -1]  # Último valor en X (extremo derecho del modelo)
presa_y = grid_y[-1, -1]  # Último valor en Y (extremo superior del modelo)
presa_z = grid_z[-1, -1] + 50  # Elevado ligeramente para visibilidad

# Verificar coordenadas de la presa
print("Coordenadas del marcador de la presa:")
print(f"Presa X: {presa_x}")
print(f"Presa Y: {presa_y}")
print(f"Presa Z: {presa_z}")

# Crear la figura interactiva
fig = go.Figure()

# Añadir superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',  # Escala de colores para representar elevaciones
    colorbar=dict(title="Elevación Z (metros)")
))

# Añadir un marcador único para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],  # Coordenada X de la presa
    y=[presa_y],  # Coordenada Y de la presa
    z=[presa_z],  # Coordenada Z de la presa (ajustada para visibilidad)
    mode='markers+text',
    marker=dict(size=12, color='blue'),  # Tamaño y color del marcador
    text=["Presa de Vajont"],  # Texto de la etiqueta
    textposition="top center",
    name="Presa"
))

# Configurar el diseño
fig.update_layout(
    title="Modelo Digital del Terreno con la Presa de Vajont",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo interactivo
fig.show()


In [ ]:
# Verificar los datos del terreno
print("Primeros valores de grid_x:")
print(grid_x[:5, :5])

print("Primeros valores de grid_y:")
print(grid_y[:5, :5])

print("Primeros valores de grid_z:")
print(grid_z[:5, :5])

# Verificar el rango del terreno
print("Rango de los datos:")
print(f"X: {grid_x.min()} a {grid_x.max()}")
print(f"Y: {grid_y.min()} a {grid_y.max()}")
print(f"Z: {grid_z.min()} a {grid_z.max()}")


In [ ]:
import plotly.graph_objects as go

# Crear figura interactiva del terreno
fig = go.Figure()

# Añadir superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',  # Escala de colores para la elevación
    colorbar=dict(title="Elevación Z (metros)")
))

# Configurar diseño básico
fig.update_layout(
    title="Modelo Digital del Terreno",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo
fig.show()


In [ ]:
presa_x = grid_x[-1, -1]  # Último valor en X
presa_y = grid_y[-1, -1]  # Último valor en Y
presa_z = grid_z[-1, -1] + 50  # Elevar un poco para visibilidad


In [ ]:
print("Coordenadas del marcador de la presa:")
print(f"Presa X: {presa_x}")
print(f"Presa Y: {presa_y}")
print(f"Presa Z: {presa_z}")


In [ ]:
# Añadir marcador único para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],  # Coordenada X de la presa
    y=[presa_y],  # Coordenada Y de la presa
    z=[presa_z],  # Coordenada Z de la presa
    mode='markers+text',
    marker=dict(size=12, color='blue'),  # Tamaño y color del marcador
    text=["Presa de Vajont"],  # Texto de la etiqueta
    textposition="top center",
    name="Presa"
))

# Mostrar el modelo actualizado
fig.show()


In [ ]:
# Generar el modelo del terreno
fig = go.Figure()

# Superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Marcador para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Configurar el diseño del gráfico
fig.update_layout(
    title="Modelo Digital del Terreno con la Presa de Vajont",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo
fig.show()


In [ ]:
import os

# Listar contenido principal del repositorio
contenido_principal = os.listdir("FW_Vajont_2019")
print("Contenido del repositorio principal:", contenido_principal)

# Listar el contenido de la carpeta DTM
contenido_dtm = os.listdir("FW_Vajont_2019/DTM")
print("Contenido de la carpeta DTM:", contenido_dtm)

# Listar el contenido de la carpeta topography
contenido_topography = os.listdir("FW_Vajont_2019/topography")
print("Contenido de la carpeta topography:", contenido_topography)


In [ ]:
# Confirmar las dimensiones y rangos de las matrices
print("Dimensiones y rangos del modelo:")
print(f"grid_x: Forma = {grid_x.shape}, Min = {grid_x.min()}, Max = {grid_x.max()}")
print(f"grid_y: Forma = {grid_y.shape}, Min = {grid_y.min()}, Max = {grid_y.max()}")
print(f"grid_z: Forma = {grid_z.shape}, Min = {grid_z.min()}, Max = {grid_z.max()}")

# Confirmar ubicación del marcador
print("Coordenadas del marcador de la presa:")
print(f"Presa X: {presa_x}")
print(f"Presa Y: {presa_y}")
print(f"Presa Z: {presa_z}")


In [ ]:
# Ajustar la ubicación del marcador
presa_x = grid_x[-10, -10]  # Coordenada cercana al borde en X
presa_y = grid_y[-10, -10]  # Coordenada cercana al borde en Y
presa_z = grid_z[-10, -10] + 10  # Elevar un poco para visibilidad
print(f"Nuevas coordenadas de la presa: ({presa_x}, {presa_y}, {presa_z})")


In [ ]:
# Crear figura interactiva
fig = go.Figure()

# Superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Marcador de la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Configurar diseño del gráfico
fig.update_layout(
    title="Modelo Digital del Terreno con Marcador de la Presa",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo actualizado
fig.show()


In [ ]:
import plotly.graph_objects as go

# Crear el modelo 3D
fig = go.Figure()

# Superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Proyección en planta (curvas de nivel)
fig.add_trace(go.Contour(
    z=grid_z,
    x=grid_x[0],
    y=grid_y[:, 0],
    contours=dict(
        coloring="lines",
        showlabels=True,
        labelfont=dict(size=10, color="black")
    ),
    line=dict(smoothing=0.85),
    name="Curvas de Nivel"
))

# Configurar título y diseño
fig.update_layout(
    title="Modelo Digital del Terreno con Perfil y Curvas de Nivel",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo
fig.show()


In [ ]:
# Crear un gráfico adicional para la sección
import matplotlib.pyplot as plt

# Generar una sección a lo largo del eje Y en un valor fijo de X (por ejemplo, el centro del modelo)
indice_x = len(grid_x) // 2  # Corte en el centro
seccion_y = grid_y[:, 0]  # Valores en el eje Y
seccion_z = grid_z[:, indice_x]  # Perfil de elevación correspondiente

# Graficar el perfil de elevación
plt.figure(figsize=(10, 6))
plt.plot(seccion_y, seccion_z, label="Perfil Topográfico")
plt.xlabel("Coordenada Y")
plt.ylabel("Elevación Z")
plt.title("Perfil Topográfico (Sección en X)")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
from ipywidgets import interact

# Función para actualizar el perfil dinámicamente
def actualizar_perfil(x_index=0):
    plt.figure(figsize=(10, 6))
    seccion_y = grid_y[:, 0]  # Valores en el eje Y
    seccion_z = grid_z[:, x_index]  # Perfil dinámico
    plt.plot(seccion_y, seccion_z, label=f"Perfil en X={grid_x[0, x_index]:.2f}")
    plt.xlabel("Coordenada Y")
    plt.ylabel("Elevación Z")
    plt.title(f"Perfil Topográfico Dinámico (X={grid_x[0, x_index]:.2f})")
    plt.grid(True)
    plt.legend()
    plt.show()

# Control interactivo
interact(actualizar_perfil, x_index=(0, grid_x.shape[1] - 1, 1))


In [ ]:
# Añadir curvas de nivel al modelo
fig.add_trace(go.Contour(
    z=grid_z[0],  # Primer plano Z (en proyección XY)
    x=grid_x[0],
    y=grid_y[:, 0],
    contours=dict(showlabels=True, labelfont=dict(size=10, color='black')),
    colorscale="Viridis",
    line_width=2,
    colorbar=dict(title="Elevación (Z)")
))

# Actualizar el diseño para las curvas
fig.update_layout(
    title="Modelo Digital del Terreno con Curvas de Nivel",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo actualizado
fig.show()


In [ ]:
!pip install dash


In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Crear la aplicación Dash
app = Dash(__name__)

# Figura 3D inicial
fig_3d = go.Figure()

fig_3d.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale="Viridis",
    colorbar=dict(title="Elevación Z (metros)")
))

fig_3d.update_layout(
    title="Modelo Digital del Terreno (Interactivo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Diseño de la aplicación
app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id="modelo-3d",
            figure=fig_3d
        )
    ], style={"width": "70%", "display": "inline-block"}),

    html.Div([
        dcc.Graph(id="perfil-dinamico")
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"})
])


# Callback para generar el perfil dinámico
@app.callback(
    Output("perfil-dinamico", "figure"),
    Input("modelo-3d", "hoverData")  # Detectar el movimiento del mouse
)
def actualizar_perfil(hover_data):
    if hover_data:
        # Extraer las coordenadas del punto donde está el mouse
        x_hover = hover_data["points"][0]["x"]
        y_hover = hover_data["points"][0]["y"]

        # Encontrar la fila correspondiente en la malla
        idx_x = (np.abs(grid_x[0] - x_hover)).argmin()
        idx_y = (np.abs(grid_y[:, 0] - y_hover)).argmin()

        # Generar el perfil
        perfil_x = grid_x[:, idx_y]  # Fila de X en la malla
        perfil_z = grid_z[:, idx_y]  # Elevación Z correspondiente

        # Crear la figura del perfil
        fig_perfil = px.line(
            x=perfil_x,
            y=perfil_z,
            labels={"x": "Coordenada X", "y": "Elevación Z"},
            title="Perfil de Elevación (XY)"
        )
        return fig_perfil

    # Si no hay datos del mouse, devolver una figura vacía
    return px.line(title="Pase el mouse sobre el modelo para ver el perfil")


# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)


In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


In [ ]:
# Crear la aplicación Dash
app = Dash(__name__)


In [ ]:
# Figura 3D inicial
fig_3d = go.Figure()

fig_3d.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale="Viridis",
    colorbar=dict(title="Elevación Z (metros)")
))

fig_3d.update_layout(
    title="Modelo Digital del Terreno (Interactivo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)


In [ ]:
# Diseño de la aplicación
app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id="modelo-3d",
            figure=fig_3d
        )
    ], style={"width": "70%", "display": "inline-block"}),

    html.Div([
        dcc.Graph(id="perfil-dinamico")
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"})
])


In [ ]:
# Callback para generar el perfil dinámico
@app.callback(
    Output("perfil-dinamico", "figure"),
    Input("modelo-3d", "hoverData")  # Detectar el movimiento del mouse
)
def actualizar_perfil(hover_data):
    if hover_data:
        # Extraer las coordenadas del punto donde está el mouse
        x_hover = hover_data["points"][0]["x"]
        y_hover = hover_data["points"][0]["y"]

        # Encontrar la fila correspondiente en la malla
        idx_x = (np.abs(grid_x[0] - x_hover)).argmin()
        idx_y = (np.abs(grid_y[:, 0] - y_hover)).argmin()

        # Generar el perfil
        perfil_x = grid_x[:, idx_y]  # Fila de X en la malla
        perfil_z = grid_z[:, idx_y]  # Elevación Z correspondiente

        # Crear la figura del perfil
        fig_perfil = px.line(
            x=perfil_x,
            y=perfil_z,
            labels={"x": "Coordenada X", "y": "Elevación Z"},
            title="Perfil de Elevación (XY)"
        )
        return fig_perfil

    # Si no hay datos del mouse, devolver una figura vacía
    return px.line(title="Pase el mouse sobre el modelo para ver el perfil")


In [ ]:
# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)


In [ ]:
@app.callback(
    Output("perfil-dinamico", "figure"),
    Input("modelo-3d", "hoverData")  # Detectar el movimiento del mouse
)
def actualizar_perfil(hover_data):
    if hover_data:
        # Extraer las coordenadas del punto donde está el mouse
        x_hover = hover_data["points"][0]["x"]

        # Encontrar la columna correspondiente en la malla
        idx_x = (np.abs(grid_x[0] - x_hover)).argmin()

        # Generar la sección topográfica a lo largo del eje Y
        perfil_y = grid_y[:, 0]  # Coordenadas en Y
        perfil_z = grid_z[:, idx_x]  # Elevación Z correspondiente a la sección

        # Crear el gráfico del perfil
        fig_perfil = px.line(
            x=perfil_y,
            y=perfil_z,
            labels={"x": "Coordenada Y", "y": "Elevación Z"},
            title="Perfil Topográfico (Sección en X)"
        )
        return fig_perfil

    # Si no hay datos del mouse, devolver una figura vacía
    return px.line(title="Pase el mouse sobre el modelo para ver el perfil")


In [ ]:
# prompt: continuar con el anteriro y corregir el error buscar en github una solucion

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
import shutil
import os
import py7zr
from scipy.interpolate import griddata
import plotly.graph_objects as go
from ipywidgets import interact
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# ... (Your existing code for data loading and preprocessing) ...

# Assuming 'grid_x', 'grid_y', and 'grid_z' are defined

# Coordenadas ajustadas de la presa (ajusta estos valores según tus datos)
presa_x = grid_x[-1, -1]  # Último valor en X (extremo derecho del modelo)
presa_y = grid_y[-1, -1]  # Último valor en Y (extremo superior del modelo)
presa_z = grid_z[-1, -1] + 50  # Elevado ligeramente para visibilidad


# Crear figura interactiva
fig = go.Figure()

# Añadir superficie del terreno
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Viridis',
    colorbar=dict(title="Elevación Z (metros)")
))

# Añadir un único marcador para la presa
fig.add_trace(go.Scatter3d(
    x=[presa_x],
    y=[presa_y],
    z=[presa_z],
    mode='markers+text',
    marker=dict(size=12, color='blue'),
    text=["Presa de Vajont"],
    textposition="top center",
    name="Presa"
))

# Configurar el diseño
fig.update_layout(
    title="Modelo Digital del Terreno con la Presa de Vajont",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Mostrar el modelo
fig.show()


# ... (Rest of your code for profile generation and interactive plot)

# Crear la aplicación Dash (corrección de la ejecución de Dash en Colab)
app = Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# ... (Your existing Dash app layout and callback code)

# Ejecutar la aplicación (modificado para Colab)
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True) # 'inline' es esencial en Google Colab

In [ ]:
# Crear las mallas grid_x, grid_y, y grid_z usando la interpolación
from scipy.interpolate import griddata

grid_x, grid_y = np.meshgrid(
    np.linspace(dtm_data['X'].min(), dtm_data['X'].max(), 100),
    np.linspace(dtm_data['Y'].min(), dtm_data['Y'].max(), 100)
)

grid_z = griddata(
    (dtm_data['X'], dtm_data['Y']),
    dtm_data['Z'],
    (grid_x, grid_y),
    method='linear'
)


In [ ]:
# Instalar las dependencias necesarias
!pip install jupyter-dash dash plotly pandas numpy scipy

# Importar las bibliotecas requeridas
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# Suponiendo que tienes un DataFrame llamado 'dtm_data' con columnas 'X', 'Y' y 'Z'

# Crear la grilla para el modelo digital del terreno
grid_x, grid_y = np.meshgrid(
    np.linspace(dtm_data['X'].min(), dtm_data['X'].max(), 100),
    np.linspace(dtm_data['Y'].min(), dtm_data['Y'].max(), 100)
)

grid_z = griddata(
    (dtm_data['X'], dtm_data['Y']),
    dtm_data['Z'],
    (grid_x, grid_y),
    method='linear'
)

# Crear la figura interactiva 3D con Plotly
fig_3d = go.Figure()

fig_3d.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale="Viridis",
    colorbar=dict(title="Elevación Z (metros)")
))

fig_3d.update_layout(
    title="Modelo Digital del Terreno (Interactivo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Crear la aplicación Dash
app = JupyterDash(__name__)

# Diseño de la aplicación
app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id="modelo-3d",
            figure=fig_3d
        )
    ], style={"width": "70%", "display": "inline-block"}),

    html.Div([
        dcc.Graph(id="perfil-dinamico")
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"})
])

# Callback para actualizar el perfil dinámico
@app.callback(
    Output("perfil-dinamico", "figure"),
    Input("modelo-3d", "hoverData")
)
def actualizar_perfil(hover_data):
    if hover_data:
        # Extraer las coordenadas del punto donde se encuentra el mouse
        x_hover = hover_data["points"][0]["x"]
        y_hover = hover_data["points"][0]["y"]

        # Encontrar los índices más cercanos en la grilla
        idx_x = (np.abs(grid_x[0, :] - x_hover)).argmin()
        idx_y = (np.abs(grid_y[:, 0] - y_hover)).argmin()

        # Generar el perfil topográfico
        perfil_x = grid_x[idx_y, :]
        perfil_z = grid_z[idx_y, :]

        # Crear la figura del perfil
        fig_perfil = px.line(
            x=perfil_x,
            y=perfil_z,
            labels={"x": "Coordenada X", "y": "Elevación Z"},
            title="Perfil de Elevación (Sección en Y)"
        )
        return fig_perfil

    # Si no hay datos del mouse, devolver una figura vacía
    return px.line(title="Pase el mouse sobre el modelo para ver el perfil")

# Ejecutar la aplicación Dash
app.run_server(mode='inline', debug=True)


In [ ]:
# Convertir las mallas en un DataFrame
modelo_df = pd.DataFrame({
    "X": grid_x.flatten(),
    "Y": grid_y.flatten(),
    "Z": grid_z.flatten()
})

# Guardar en un archivo CSV
modelo_df.to_csv("modelo_terreno.csv", index=False)
print("Modelo guardado en 'modelo_terreno.csv'.")


In [ ]:
# Guardar la figura interactiva como archivo HTML
fig_3d.write_html("modelo_terreno_interactivo.html")
print("Modelo interactivo guardado como 'modelo_terreno_interactivo.html'.")


In [ ]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Crear la aplicación Dash
app = Dash(__name__)

# Figura 3D inicial
fig_3d = go.Figure()

fig_3d.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale="Viridis",
    colorbar=dict(title="Elevación Z (metros)")
))

fig_3d.update_layout(
    title="Modelo Digital del Terreno (Interactivo)",
    scene=dict(
        xaxis_title="Coordenada X",
        yaxis_title="Coordenada Y",
        zaxis_title="Elevación Z"
    )
)

# Diseño de la aplicación
app.layout = html.Div([
    html.H3("Pase el mouse sobre el modelo y haga clic para fijar un punto."),

    html.Div([
        dcc.Graph(
            id="modelo-3d",
            figure=fig_3d,
            config={"clickmode": "event+select"}  # Permitir clics en el modelo
        )
    ], style={"width": "70%", "display": "inline-block"}),

    html.Div([
        dcc.Graph(id="perfil-dinamico"),
        html.Div(id="propiedades-punto", style={"marginTop": "20px"})  # Mostrar propiedades del punto
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"})
])

# Callback para manejar clics en el modelo
@app.callback(
    [Output("perfil-dinamico", "figure"),
     Output("propiedades-punto", "children")],  # Actualizar gráfico y propiedades del punto
    [Input("modelo-3d", "clickData")],          # Detectar clic en el modelo
    [State("perfil-dinamico", "figure")]        # Mantener el estado del gráfico
)
def fijar_punto(click_data, perfil_actual):
    if click_data:
        # Extraer coordenadas del punto seleccionado
        x_click = click_data["points"][0]["x"]
        y_click = click_data["points"][0]["y"]

        # Buscar el índice más cercano en la malla
        idx_x = (np.abs(grid_x[0, :] - x_click)).argmin()
        idx_y = (np.abs(grid_y[:, 0] - y_click)).argmin()

        # Generar el perfil en la dirección Y
        perfil_x = grid_x[idx_y, :]
        perfil_z = grid_z[idx_y, :]

        # Crear el gráfico del perfil
        fig_perfil = px.line(
            x=perfil_x,
            y=perfil_z,
            labels={"x": "Coordenada X", "y": "Elevación Z"},
            title="Perfil de Elevación (Sección en Y)"
        )

        # Propiedades del punto seleccionado
        propiedades = f"""
        Coordenadas del Punto Fijado:
        - X: {x_click:.2f}
        - Y: {y_click:.2f}
        - Z: {grid_z[idx_y, idx_x]:.2f}
        """

        return fig_perfil, propiedades

    # Si no hay clic, mostrar un mensaje inicial
    return px.line(title="Pase el mouse sobre el modelo y haga clic para fijar un punto"), "No se ha fijado ningún punto."

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
